# AR-SSL4M Pretraining on Google Colab

This notebook handles the setup and pretraining of the AR-SSL4M model using data from Google Drive.

In [1]:
# 代码块0 Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# ==========================================
# 代码块1
# 1. 安装核心网络组件 & 深度学习依赖
# ==========================================
print("--- 正在安装 Tailscale & S3 认证组件 ---")

# 安装 Tailscale 官方客户端
!curl -fsSL https://tailscale.com/install.sh | sh

# 安装 Python 网络请求、S3 协议及 SOCKS 代理支持
!pip install boto3 botocore PySocks requests -q

# 安装 AR-SSL4M 模型运行所需的依赖库
!pip install timm monai transformers fire -q

# ==========================================
# 2. 清理环境并启动网络堆栈
# ==========================================
import os
import time

print("\n--- 正在重置 Tailscale 网络环境 ---")
# 清理可能存在的残留进程
!pkill -9 tailscaled 2>/dev/null
!rm -rf /var/run/tailscale/tailscaled.pid

# 启动 Tailscale 守护进程并开启 SOCKS5 代理入口 (1055 端口)
!nohup tailscaled --tun=userspace-networking --socks5-server=localhost:1055 --state=/var/lib/tailscale/tailscaled.state > /tmp/tailscale.log 2>&1 &
time.sleep(5)

# ==========================================
# 3. 隧道认证 (请点击生成的链接)
# ==========================================
print("\n--- 请点击下方链接进行 Tailscale 认证 ---")
# 执行认证上线
!tailscale up --accept-dns=false

print("\n✅ 环境准备就绪！")

--- 正在安装 Tailscale & S3 认证组件 ---
Installing Tailscale for ubuntu jammy, using method apt
+ mkdir -p --mode=0755 /usr/share/keyrings
+ curl -fsSL https://pkgs.tailscale.com/stable/ubuntu/jammy.noarmor.gpg
+ tee /usr/share/keyrings/tailscale-archive-keyring.gpg
+ chmod 0644 /usr/share/keyrings/tailscale-archive-keyring.gpg
+ curl -fsSL https://pkgs.tailscale.com/stable/ubuntu/jammy.tailscale-keyring.list
+ tee /etc/apt/sources.list.d/tailscale.list
# Tailscale packages for ubuntu jammy
deb [signed-by=/usr/share/keyrings/tailscale-archive-keyring.gpg] https://pkgs.tailscale.com/stable/ubuntu jammy main
+ chmod 0644 /etc/apt/sources.list.d/tailscale.list
+ apt-get update
Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu220

In [3]:
# 代码块2
#  验证预签名 URL 列表文件 (不再检查本地文件夹，改为检查 URL 清单)
import os

# 这是你存放在 Google Drive 里的全量 URL 列表路径
urls_list_path = '/content/drive/MyDrive/dataset/pretrain/DeepLesion/urls.txt'

if os.path.exists(urls_list_path):
    print(f"✅ 找到 URL 列表文件: {urls_list_path}")

    with open(urls_list_path, 'r') as f:
        all_lines = f.readlines()

    # 过滤掉元数据，只统计真正的数据链接
    valid_urls = [line.strip() for line in all_lines if '.npy?' in line and '.minio.sys' not in line]

    print(f"📊 列表验证结果:")
    print(f"   - 总行数: {len(all_lines)}")
    print(f"   - 有效 .npy 链接数: {len(valid_urls)}")

    if len(valid_urls) > 0:
        print(f"🔗 样本预览: {valid_urls[0][:100]}...") # 只打印前100字符防止刷屏
    else:
        print("❌ 警告：列表中未发现有效的 .npy 链接！请检查 Windows 端的生成逻辑。")
else:
    print(f"❌ 错误：在 {urls_list_path} 未找到文件。")
    print("请确认你已将 Windows 生成的 urls.txt 上传至该 Drive 路径。")

✅ 找到 URL 列表文件: /content/drive/MyDrive/dataset/pretrain/DeepLesion/urls.txt
📊 列表验证结果:
   - 总行数: 9818
   - 有效 .npy 链接数: 9816
🔗 样本预览: http://100.77.8.124:9000/nnpy/npy/000001_01_01_1_3.npy?AWSAccessKeyId=admin&Signature=B1DSa%2Bc%2B5B...


In [4]:
# 代码块3
# Data Verification (Tailscale Online Mode)
# 不再全局校验文件，改为随机抽样测试隧道链路和数据格式
import os
import requests
import numpy as np
import io
import random
from tqdm import tqdm

urls_list_path = '/content/drive/MyDrive/dataset/pretrain/DeepLesion/urls.txt'
sample_size = 10  # 抽样测试 10 个文件即可，无需全部校验

# 代理配置
proxies = {
    'http': 'socks5h://127.0.0.1:1055',
    'https': 'socks5h://127.0.0.1:1055'
}

if os.path.exists(urls_list_path):
    with open(urls_list_path, 'r') as f:
        urls = [line.strip() for line in f if '.npy?' in line and '.minio.sys' not in line]

    print(f"--- 正在对 {len(urls)} 个远程文件进行随机抽样校验 ({sample_size}个) ---")

    # 随机抽样
    test_samples = random.sample(urls, min(sample_size, len(urls)))

    success_count = 0
    for url in tqdm(test_samples):
        try:
            # 通过隧道拉取数据到内存
            r = requests.get(url, proxies=proxies, timeout=15)
            r.raise_for_status()

            # 验证是否为有效的 numpy 数据
            data = np.load(io.BytesIO(r.content))

            # 验证维度 (参考原代码 128x128x32)
            if data.shape == (128, 128, 32):
                success_count += 1
            else:
                print(f"\n⚠️ 维度不匹配: {data.shape}, URL: {url[:50]}...")

        except Exception as e:
            print(f"\n❌ 校验失败: {e}")

    print(f"\n--- 校验完成 ---")
    print(f"✅ 成功通过测试: {success_count}/{len(test_samples)}")

    if success_count == len(test_samples):
        print("🚀 隧道链路与数据格式完全正常，可以开始训练。")
    else:
        print("🛑 链路不稳定或数据有误，请检查 Tailscale 状态。")
else:
    print(f"❌ 找不到列表文件: {urls_list_path}")

--- 正在对 9816 个远程文件进行随机抽样校验 (10个) ---


100%|██████████| 10/10 [00:10<00:00,  1.08s/it]


--- 校验完成 ---
✅ 成功通过测试: 10/10
🚀 隧道链路与数据格式完全正常，可以开始训练。


In [5]:
# 代码块4
# 1. 克隆代码仓库并进入工作目录
project_root = '/content/AR-SSL4M-DEMO'

# 如果目录已存在，先删除（确保每次都是从 GitHub 拉取最新代码）
if os.path.exists(project_root):
    import shutil
    shutil.rmtree(project_root)

# 克隆你的仓库
!git clone https://github.com/tanglehunter00/AR-SSL4M-DEMO.git

import os
if os.path.exists(project_root):
    # 切换工作目录到 newPretrain 文件夹，方便后续运行 main.py
    %cd {project_root}/newPretrain
    print(f"✅ 已进入项目根目录: {os.getcwd()}")
else:
    print("❌ 错误：仓库克隆失败，请检查网络或 GitHub 链接。")

Cloning into 'AR-SSL4M-DEMO'...
remote: Enumerating objects: 329, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 329 (delta 31), reused 24 (delta 15), pack-reused 274 (from 1)
Receiving objects: 100% (329/329), 1.70 MiB | 38.67 MiB/s, done.
Resolving deltas: 100% (151/151), done.
/content/AR-SSL4M-DEMO/newPretrain
✅ 已进入项目根目录: /content/AR-SSL4M-DEMO/newPretrain


In [6]:
# 代码块5
# Install dependencies
!pip install timm monai transformers fire

In [7]:
# 代码块6
# Update dataset configuration to use Online URLs
# 不再扫描 Drive 文件夹，直接将 urls.txt 作为数据源列表

import os

# 1. 定义 URL 列表的来源（来自你的 Google Drive）
urls_list_source = '/content/drive/MyDrive/dataset/pretrain/DeepLesion/urls.txt'

# 2. 定义训练时使用的列表路径（保持与原代码变量名一致，防止 main.py 报错）
list_file_path = urls_list_source

if os.path.exists(urls_list_source):
    # 读取并简单验证列表行数
    with open(urls_list_source, 'r') as f:
        url_count = len([line for line in f if '.npy?' in line])

    print(f"✅ 数据配置已更新！")
    print(f"🔗 训练列表指向: {list_file_path}")
    print(f"📄 检测到有效数据链接: {url_count} 条")
    print("💡 提示：系统将通过这些预签名 URL 直接从你的 PC 抓取数据，不再占用 Colab 硬盘。")
else:
    print(f"❌ 错误：在 {urls_list_source} 未找到 URL 列表。")
    print("请确认你已将 Windows 生成的 urls.txt 上传至该 Drive 路径。")

✅ 数据配置已更新！
🔗 训练列表指向: /content/drive/MyDrive/dataset/pretrain/DeepLesion/urls.txt
📄 检测到有效数据链接: 9816 条
💡 提示：系统将通过这些预签名 URL 直接从你的 PC 抓取数据，不再占用 Colab 硬盘。


In [ ]:
# 代码块7
# 修改 datasets.py 配置文件，将数据路径指向 URL 列表
# 注意：此时我们已经在 /content/AR-SSL4M-DEMO/newPretrain 目录下

import os
from typing import Optional

# 修正相对路径：因为当前已在 pretrain 文件夹内
config_path = 'configs/datasets.py'

# 注入配置信息
new_config_content = f"""
from dataclasses import dataclass

@dataclass
class custom_dataset:
    dataset: str = "custom_dataset"
    file: str = "image_dataset.py"
    train_split: str = "train"
    test_split: str = "validation"
    # 这里 list_file_path 指向的是包含预签名 URL 的 urls.txt
    spatial_path: str = "{list_file_path}"
    contrast_path: str = "{list_file_path}"
    semantic_path: str = "{list_file_path}"
    fetch_proxy: str = "socks5h://127.0.0.1:1055"
    img_size = [128, 128, 32]
    patch_size = [16, 16, 4]
    attention_type = 'prefix'
    add_series_data = False
    add_spatial_data = True
    is_subset = False
    series_length = 4
"""

# 执行写入
with open(config_path, 'w') as f:
    f.write(new_config_content)

print(f"✅ 成功更新 {config_path}")
print(f"🚀 训练将从以下 URL 列表拉取数据: {list_file_path}")

✅ 成功更新 configs/datasets.py
🚀 训练将从以下 URL 列表拉取数据: /content/drive/MyDrive/dataset/pretrain/DeepLesion/urls.txt


In [ ]:
# ==========================================
# 代码块8
# 最终步骤：启动在线预训练
# ==========================================

import os

# 1. 确保输出目录存在 (用于保存权重和 Metrics)
output_dir = "/content/drive/MyDrive/dataset/LIDC-IDRI/output"
os.makedirs(output_dir, exist_ok=True)

print(f"--- 🚀 正在启动 AR-SSL4M 在线预训练 ---")
print(f"数据源: {list_file_path}")
print(f"输出路径: {output_dir}")

# 2. 运行训练 (针对在线模式优化参数)
# --batch_size_training: 建议根据 A100 显存和网络带宽设为 16 或 32，原代码 128 对中继压力太大
# --num_workers_dataloader: 必须调低！设为 2 可在下载并发与稳定性之间取得平衡
# --enable_fsdp: 设为 False，单卡 A100 无需分片

!python main.py \
    --enable_fsdp False \
    --output_dir {output_dir} \
    --use_prefetch 1 \
    --prefetch_cache_dir /tmp/ar_ssl4m_prefetch_cache \
    --prefetch_buffer_batches 3 \
    --batch_size_training 64 \
    --num_epochs 5 \
    --save_metrics True

--- 🚀 正在启动 AR-SSL4M 在线预训练 ---
数据源: /content/drive/MyDrive/dataset/pretrain/DeepLesion/urls.txt
输出路径: /content/drive/MyDrive/dataset/LIDC-IDRI/output
